In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split 

In [3]:
df = pd.read_csv('sredjeni_kurs.csv', parse_dates=True)

In [4]:
# Pretvaranje podataka u vremensku seriju
time_series = df['MIDDLE EXCHANGE RATE'].values.astype(float)

In [5]:
# Normalizacija podataka
scaler = MinMaxScaler(feature_range=(0, 1))
time_series_normalized = scaler.fit_transform(time_series.reshape(-1, 1))

In [6]:
# Funkcija za pretvaranje vremenske serije u uzorke i ciljeve
def create_dataset(time_series, time_steps):
    X, y = [], []
    for i in range(len(time_series) - time_steps):
        X.append(time_series[i:(i + time_steps)])
        y.append(time_series[i + time_steps])
    return np.array(X), np.array(y)

In [7]:
# Postavljanje parametara
time_steps = 30  # Broj koraka vremenske serije za svaki uzorak
batch_size = 64
epochs = 100

# Kreiranje uzoraka i ciljeva
X, y = create_dataset(time_series_normalized, time_steps)

In [10]:
# Podela podataka na skupove za obuku i testiranje
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Kreiranje modela
model = Sequential([
    LSTM(50, input_shape=(time_steps, 1)),
    Dense(1)
])

# Kompilacija modela
model.compile(optimizer='adam', loss='mean_squared_error')

# Treniranje modela
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/100
96/96 [==============================] - 3s 13ms/step - loss: 0.0245
Epoch 2/100
96/96 [==============================] - 1s 12ms/step - loss: 1.0983e-04
Epoch 3/100
96/96 [==============================] - 1s 12ms/step - loss: 8.1071e-05
Epoch 4/100
96/96 [==============================] - 1s 12ms/step - loss: 7.8076e-05
Epoch 5/100
96/96 [==============================] - 1s 12ms/step - loss: 7.5237e-05
Epoch 6/100
96/96 [==============================] - 1s 12ms/step - loss: 7.3145e-05
Epoch 7/100
96/96 [==============================] - 1s 12ms/step - loss: 7.2568e-05
Epoch 8/100
96/96 [==============================] - 1s 12ms/step - loss: 7.2039e-05
Epoch 9/100
96/96 [==============================] - 1s 12ms/step - loss: 7.0714e-05
Epoch 10/100
96/96 [==============================] - 1s 12ms/step - loss: 7.0800e-05
Epoch 11/100
96/96 [==============================] - 1s 12ms/step - loss: 7.0006e-05
Epoch 12/100
96/96 [==============================] - 1s 12ms/step 

96/96 [==============================] - 1s 12ms/step - loss: 2.9038e-05
Epoch 97/100
96/96 [==============================] - 1s 12ms/step - loss: 2.6265e-05
Epoch 98/100
96/96 [==============================] - 1s 12ms/step - loss: 2.5839e-05
Epoch 99/100
96/96 [==============================] - 1s 12ms/step - loss: 2.3647e-05
Epoch 100/100
96/96 [==============================] - 1s 12ms/step - loss: 2.3326e-05


In [12]:
# Evaluacija modela
loss = model.evaluate(X_test, y_test)
print('Test Loss:', loss)


48/48 [==============================] - 1s 6ms/step - loss: 2.1432e-05
Test Loss: 2.1432017092593014e-05


In [13]:
# Poslednjih 30 vremenskih koraka iz testnog skupa
X_future = X_test[-30:]

# Predviđanje kretanja kursa dinara za budući period
future_predictions_normalized = model.predict(X_future)
future_predictions = scaler.inverse_transform(future_predictions_normalized)
print('Future Predictions:', future_predictions)

1/1 [==============================] - 0s 278ms/step
Future Predictions: [[117.69721 ]
 [ 65.14468 ]
 [106.1674  ]
 [ 87.625305]
 [117.667175]
 [118.165886]
 [113.68252 ]
 [117.29291 ]
 [ 71.494896]
 [122.9822  ]
 [ 97.01296 ]
 [ 93.27349 ]
 [113.910965]
 [120.832245]
 [120.65601 ]
 [117.54235 ]
 [ 77.58454 ]
 [123.38518 ]
 [ 81.65068 ]
 [117.70015 ]
 [118.379135]
 [118.08447 ]
 [ 78.78515 ]
 [117.68933 ]
 [118.045   ]
 [ 86.76111 ]
 [ 94.770706]
 [117.50257 ]
 [ 80.82784 ]
 [ 65.34538 ]]


In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Izračunaj predviđanja za testni skup podataka
predictions = model.predict(X_test)

# Izračunaj MAE
mae = mean_absolute_error(y_test, predictions)

# Izračunaj MSE
mse = mean_squared_error(y_test, predictions)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)

48/48 [==============================] - 0s 6ms/step
Mean Absolute Error (MAE): 0.0028588411836651508
Mean Squared Error (MSE): 2.1432012656953045e-05
